In [1]:
import torch

In [2]:
B = 32
T = 40
D_dec = 300
D_enc = 150
N = 4

In [3]:
Z = torch.randn(N, D_dec, D_enc)
enc = torch.randn(B, T + 1, D_enc)
dec = torch.randn(B, T, D_dec)

In [25]:
my_sln = (dec[:, :, None, None, None, :] @ (Z[None, None, :, :, :] @ enc[:, None, :, None, :, None])).squeeze(-1, -2)

In [5]:
my_sln.shape

torch.Size([32, 40, 41, 4])

In [6]:
einsum_step_1 = torch.einsum("nij,bsj->bsni", Z, enc)

In [7]:
einsum_step_1.shape

torch.Size([32, 41, 4, 300])

In [8]:
einsum_step_2 = torch.einsum("bti,bsni->btsn", dec, einsum_step_1)

In [9]:
einsum_step_2.shape

torch.Size([32, 40, 41, 4])

In [10]:
(torch.isclose(my_sln, einsum_step_2, rtol=0.001) == False).sum() / einsum_step_1.numel()

tensor(2.0325e-05)

In [11]:
einsum = torch.einsum("nij,bsj,bti->btsn", Z, enc, dec)

In [12]:
einsum.shape

torch.Size([32, 40, 41, 4])

In [13]:
(torch.isclose(my_sln, einsum, rtol=0.001) == False).sum() / einsum.numel()

tensor(0.0002)

In [181]:
(torch.isclose(einsum, einsum_step_2, rtol=0.001) == False).sum() / einsum.numel()

tensor(2.3819e-05)

In [182]:
einsum.shape

torch.Size([32, 40, 41, 4])